# Data Cleaning and Inspection
## Columns
- **work_year**: Year of salary data
- **experience_leavel**: Employee seniority level (e.g., Junior, Mid, Senior)
- **employment_type**: Full time or part time
- **job_title**: Position held
- **salary**: Salary in local currency
- **salary_currency**: Local currenct
- **salary_in_usd**: Salary converted to USD
- **employee_residence**: Country employee lives in
- **remote_ratio**: Ratio of time worked remotely
- **company_location**: Country of company
- **company_size**: Saize of country (e.g, Small, Medium, Large)


In [57]:
#Imports
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
from helper_functions import create_key, merge_df

#Pull down dataframe
df = pd.read_csv("./data/Dataset salary 2024.csv")

## Data Inpection

In [32]:
#Zero null values
df.isna().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [33]:
#Summary stats
pd.set_option("display.float_format", "{:,.2f}".format)
df.describe()

,work_year,salary,salary_in_usd,remote_ratio
count,"16,534.00","16,534.00","16,534.00","16,534.00"
mean,"2,023.23","163,726.96","149,686.78",32.00
std,0.71,"340,205.74","68,505.29",46.25
min,"2,020.00","14,000.00","15,000.00",0.00
25%,"2,023.00","101,763.00","101,125.00",0.00
50%,"2,023.00","142,200.00","141,300.00",0.00
75%,"2,024.00","187,200.00","185,900.00",100.00
max,"2,024.00","30,400,000.00","800,000.00",100.00


In [34]:
#Columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16534 entries, 0 to 16533
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           16534 non-null  int64 
 1   experience_level    16534 non-null  object
 2   employment_type     16534 non-null  object
 3   job_title           16534 non-null  object
 4   salary              16534 non-null  int64 
 5   salary_currency     16534 non-null  object
 6   salary_in_usd       16534 non-null  int64 
 7   employee_residence  16534 non-null  object
 8   remote_ratio        16534 non-null  int64 
 9   company_location    16534 non-null  object
 10  company_size        16534 non-null  object
dtypes: int64(4), object(7)
memory usage: 1.4+ MB


In [35]:
#Rows x Columns
df.shape

(16534, 11)

In [36]:
#Different Job titles
df["job_title"].unique()

array(['AI Engineer', 'Data Engineer', 'Machine Learning Engineer',
       'ML Engineer', 'Data Analyst', 'NLP Engineer', 'Data Scientist',
       'Applied Scientist', 'Data Manager', 'Data Architect',
       'Data Operations Manager', 'Research Scientist',
       'Data Science Manager', 'Business Intelligence Engineer',
       'Business Intelligence', 'Data Modeler',
       'Data Integration Specialist', 'Data Product Manager',
       'Data Visualization Specialist', 'Data Science', 'Data Specialist',
       'Business Intelligence Lead', 'Research Analyst',
       'Business Intelligence Analyst', 'AI Software Engineer',
       'Machine Learning Scientist', 'Data Science Engineer',
       'Machine Learning Operations Engineer', 'Analytics Engineer',
       'Data Management Analyst',
       'Encounter Data Management Professional',
       'Data Analytics Specialist', 'Head of Data', 'Data Analytics Lead',
       'Research Engineer', 'AI Scientist',
       'Business Intelligence Develope

## Data Cleaning

### First let's make the data frames into their respective tables in the designed schema

In [49]:
employee_df = df[["work_year", "employment_type", "job_title", "salary", "salary_currency", "salary_in_usd", "remote_ratio", "employee_residence", "company_size", "company_location"]].copy()

In [50]:
#Create a Primary Key column for each employee
employee_df['employee_id'] = create_key(employee_df)
employee_df

,work_year,employment_type,job_title,salary,salary_currency,salary_in_usd,remote_ratio,employee_residence,company_size,company_location,employee_id
0,2024,FT,AI Engineer,202730,USD,202730,0,US,M,US,1
1,2024,FT,AI Engineer,92118,USD,92118,0,US,M,US,2
2,2024,FT,Data Engineer,130500,USD,130500,0,US,M,US,3
3,2024,FT,Data Engineer,96000,USD,96000,0,US,M,US,4
4,2024,FT,Machine Learning Engineer,190000,USD,190000,0,US,M,US,5
...,...,...,...,...,...,...,...,...,...,...,...
16529,2020,FT,Data Scientist,412000,USD,412000,100,US,L,US,16530
16530,2021,FT,Principal Data Scientist,151000,USD,151000,100,US,L,US,16531
16531,2020,FT,Data Scientist,105000,USD,105000,100,US,S,US,16532
16532,2020,CT,Business Data Analyst,100000,USD,100000,100,US,L,US,16533


In [51]:
#Grab unique countries
countries_company = df["company_location"].unique()
countries_employee = employee_df["employee_residence"].unique()

countries = np.concat((countries_company, countries_employee))
countries = np.unique(countries)
country_df = pd.DataFrame(countries, columns=["country_code"])


In [52]:
#Create Primary Key column for each country
country_df['country_id'] = create_key(country_df)
country_df

,country_code,country_id
0,AD,1
1,AE,2
2,AM,3
3,AR,4
4,AS,5
...,...,...
85,UG,86
86,US,87
87,UZ,88
88,VN,89


In [53]:
#Company Dataframe
company_df = df[["company_size", "company_location"]].drop_duplicates()
company_df['company_id'] = create_key(company_df)
company_df

,company_size,company_location,company_id
0,M,US,1
18,L,US,2
40,S,AU,3
48,M,GB,4
50,M,CA,5
...,...,...,...
16509,S,AS,147
16510,S,AT,148
16523,S,HR,149
16525,M,IL,150


In [54]:
#Create our relationsips
#Start with employee table
employee_df = merge_df(employee_df, country_df, ["country_code", "country_id"], ["employee_residence"], ["country_code"])
employee_df.drop(columns=["employee_residence", "country_code"], inplace=True)

employee_df = merge_df(employee_df, company_df, ["company_location", "company_size", "company_id"], ["company_location", "company_size"], ["company_location", "company_size"])
employee_df.drop(columns=["company_location", "company_size"], inplace=True)

employee_df


,work_year,employment_type,job_title,salary,salary_currency,salary_in_usd,remote_ratio,employee_id,country_id,company_id
0,2024,FT,AI Engineer,202730,USD,202730,0,1,87,1
1,2024,FT,AI Engineer,92118,USD,92118,0,2,87,1
2,2024,FT,Data Engineer,130500,USD,130500,0,3,87,1
3,2024,FT,Data Engineer,96000,USD,96000,0,4,87,1
4,2024,FT,Machine Learning Engineer,190000,USD,190000,0,5,87,1
...,...,...,...,...,...,...,...,...,...,...
16529,2020,FT,Data Scientist,412000,USD,412000,100,16530,87,2
16530,2021,FT,Principal Data Scientist,151000,USD,151000,100,16531,87,2
16531,2020,FT,Data Scientist,105000,USD,105000,100,16532,87,61
16532,2020,CT,Business Data Analyst,100000,USD,100000,100,16533,87,2


In [55]:
#Now lets do the company table
company_df = merge_df(company_df, country_df, ["country_code", "country_id"], ["company_location"], ["country_code"])
company_df.drop(columns=["country_code", "company_location"], inplace=True)

company_df

,company_size,company_id,country_id
0,M,1,87
1,L,2,87
2,S,3,7
3,M,4,33
4,M,5,14
...,...,...,...
146,S,147,5
147,S,148,6
148,S,149,40
149,M,150,44


In [56]:
#Migrate over to sql

#Load env variables
load_dotenv() 
DATABASE_URL = os.getenv("DATABASE_URL")

#Create engine
engine = create_engine(DATABASE_URL)

#We need to rename some columns to match the table...
employee_df = employee_df.rename(columns={
    "employee_id": "employeeid",
    "work_year": "year",
    "salary_currency": "currency",
    "salary_in_usd": "salary_usd",
    "country_id": "countryid",
    "company_id": "companyid"
})

company_df = company_df.rename(columns={
    "company_id": "companyid",
    "country_id": "countryid",
    "company_size": "size",
})
country_df = country_df.rename(columns={
    "country_id": "countryid",
    "country_code": "name",
})

country_df.to_sql("country", engine, if_exists="append", index=False)
company_df.to_sql("company", engine, if_exists="append", index=False)
employee_df.to_sql("employee", engine, if_exists="append", index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "country_pkey"
DETAIL:  Key (countryid)=(1) already exists.

[SQL: INSERT INTO country (name, countryid) VALUES (%(name__0)s, %(countryid__0)s), (%(name__1)s, %(countryid__1)s), (%(name__2)s, %(countryid__2)s), (%(name__3)s, %(countryid__3)s), (%(name__4)s, %(countryid__4)s), (%(name__5)s, %(countryid__5)s), (%(name ... 2823 characters truncated ... name__87)s, %(countryid__87)s), (%(name__88)s, %(countryid__88)s), (%(name__89)s, %(countryid__89)s)]
[parameters: {'countryid__0': 1, 'name__0': 'AD', 'countryid__1': 2, 'name__1': 'AE', 'countryid__2': 3, 'name__2': 'AM', 'countryid__3': 4, 'name__3': 'AR', 'countryid__4': 5, 'name__4': 'AS', 'countryid__5': 6, 'name__5': 'AT', 'countryid__6': 7, 'name__6': 'AU', 'countryid__7': 8, 'name__7': 'BA', 'countryid__8': 9, 'name__8': 'BE', 'countryid__9': 10, 'name__9': 'BG', 'countryid__10': 11, 'name__10': 'BO', 'countryid__11': 12, 'name__11': 'B